In [1]:
from __future__ import annotations
import logging
import os
import sys

import datasets as nlp_datasets
import pandas as pd
from sklearn.metrics import f1_score

from cappr import openai
sys.path.insert(1, os.path.join(sys.path[0], "..", ".."))
from utils import display_df

Another tough one

In [2]:
## When hitting the OpenAI endpoints, we'll log any server errors
logging.basicConfig(level=logging.INFO,
                    handlers=[logging.StreamHandler(stream=sys.stdout)],
                    format='%(asctime)s :: %(name)s :: %(levelname)s :: '
                           '%(message)s')
logger = logging.getLogger(__name__)

In [3]:
df = pd.DataFrame(nlp_datasets
                  .load_dataset('ought/raft', 'systematic_review_inclusion',
                                split='train'))

2023-04-04 17:19:27,286 :: datasets.builder :: WARNING :: Found cached dataset raft (C:/Users/kushd/.cache/huggingface/datasets/ought___raft/systematic_review_inclusion/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84)


In [4]:
len(df)

50

In [5]:
df.head()

,Title,Abstract,Authors,Journal,ID,Label
0,Prototyping and transforming facial textures f...,Wavelet based methods for prototyping facial t...,"Tiddeman, B.; Burt, M.; Perrett, D.",IEEE Comput Graphics Appl,0,2
1,School finance reform and voluntary fiscal fed...,California has transferred the financing of it...,"Brunner, E.; Sonstelie, J.",J. Public Econ.,1,2
2,When Should the Ask Be a Nudge? The Effect of ...,,"Goswami, I.; Urminsky, O.",Journal of Marketing Research,2,2
3,Intra-organizational volunteerism: Good soldie...,Despite the millions of hours donated to chari...,"Peloza, J.; Hassay, D.N.",J. Bus. Ethics,3,2
4,Implicit vs. Explicit deception in ultimatum g...,"We explore bargaining, using ultimatum games, ...","Kriss, P.H.; Nagel, R.; Weber, R.A.",J. Econ. Behav. Organ.,4,2


In [6]:
def prompt(title: str, abstract: str) -> str:
    return ( 'Identify whether this paper should be included in a meta-review which '
             'includes the findings of systematic reviews on interventions designed to '
             'promote charitable donations. Included reviews should describe monetary '
             'charitable donations, assess any population of participants in any '
             'context, and be peer reviewed and written in English. They should not '
             'report new data, be non-systematic reviews, consider cause-related '
             'marketing or other kinds of prosocial behaviour.\n\n'
             "Here's information about the paper.\n"
            f'Title: {title}\n'
            f'Abstract: {abstract}\n\n'
             'Should this paper be included in the meta-review described above? '
             'Answer Yes or No:')

In [7]:
df['prompt'] = [prompt(title, abstract)
                for title, abstract in zip(df['Title'], df['Abstract'])]

In [8]:
display_df(df, columns=['prompt', 'Label'], num_rows=1)

,prompt,Label
0,"Identify whether this paper should be included in a meta-review which includes the findings of systematic reviews on interventions designed to promote charitable donations. Included reviews should describe monetary charitable donations, assess any population of participants in any context, and be peer reviewed and written in English. They should not report new data, be non-systematic reviews, consider cause-related marketing or other kinds of prosocial behaviour. Here's information about the paper. Title: Prototyping and transforming facial textures for perception research Abstract: Wavelet based methods for prototyping facial textures for artificially transforming the age of facial images were described. Prototype images were used to define the salient features of a particular face classification. Two experiments were conducted to validate the ability of wavelet processing method to capture age information. The first experiment validated the textured prototyping method while the second experiment investigated the effectiveness of the new age transformation technique. The shape and color transformation used to rejuvenate faces hardly affected the apparent age. The average hair color change during rejuvenation was not sufficient to project the hair color in normal range for the younger age group. Should this paper be included in the meta-review described above? Answer Yes or No:",2


In [9]:
prior = (df['Label']
         .value_counts(normalize=True)
         .sort_index()
         .to_numpy())
prior

array([0.02, 0.98])

In [10]:
## $0.59
pred_probs = (openai.classify
              .predict_proba(df['prompt'].tolist(),
                             completions=('Yes', 'No'),
                             model='text-davinci-003',
                             prior=None,
                             ask_if_ok=True))

log-probs:   0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
f1_score(df['Label']-1, pred_probs.argmax(axis=1), average='macro')

0.4897959183673469